## Step 1: Prepare the Training and Evaluation Dataset

In [1]:
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)
(X.shape, y.shape)

((442, 10), (442,))

In [2]:
[(X[0], y[0]), (X[1], y[1])]

[(array([ 0.03807591,  0.05068012,  0.06169621,  0.02187239, -0.0442235 ,
         -0.03482076, -0.04340085, -0.00259226,  0.01990749, -0.01764613]),
  151.0),
 (array([-0.00188202, -0.04464164, -0.05147406, -0.02632753, -0.00844872,
         -0.01916334,  0.07441156, -0.03949338, -0.06833155, -0.09220405]),
  75.0)]

## Step 2: Launching the Spark Session with Rikai Extension

In [3]:
from example import spark

23/01/09 21:24:05 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.31.194 instead (on interface wlx1cbfce3ffbfe)
23/01/09 21:24:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/01/09 21:24:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/09 21:24:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/09 21:24:11 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Step 3: Training and Logging using MLflow

In [4]:
import getpass

import mlflow
from liga.sklearn.mlflow import log_model
from sklearn.linear_model import LinearRegression


mlflow_tracking_uri = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(mlflow_tracking_uri)

# train a model
model = LinearRegression()
with mlflow.start_run() as run:
    ####
    # Part 1: Train the model and register it on MLflow
    ####
    model.fit(X, y)

    registered_model_name = f"{getpass.getuser()}_sklearn_lr"
    log_model(model, registered_model_name)


/home/da/.cache/pants/named_caches/pex_root/venvs/5818e9d8e430c4195a8483476276c87a1d17cfbe/9c2b3715a935801e4ce4138ccb7afcc4a7b4d59e/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2023/01/09 21:24:14 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'da_sklearn_lr' already exists. Creating a new version of this model...
2023/01/09 21:24:14 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: da_sklearn_lr, version 24
Created version '24' of model 'da_sklearn_lr'.
/tmp/pants-sandbox-sc9D8O/python/liga/mlflow/logger.py:144: UserWarning: value of rikai.output.schema is None or empty and will not be populated to MLflow
  warnings.warn(


## Step 4: Create the model using the registered MLflow uri

In [5]:
from liga.mlflow.logger import CONF_MLFLOW_TRACKING_URI
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set(CONF_MLFLOW_TRACKING_URI, mlflow_tracking_uri)
spark.sql(f"""
CREATE OR REPLACE MODEL mlflow_sklearn_m USING 'mlflow:///{registered_model_name}';
"""
)

spark.sql("show models").show(1, vertical=False, truncate=False)


Py4JJavaError: An error occurred while calling o32.sql.
: java.lang.RuntimeException: Traceback (most recent call last):  File "<string>", line 5, in <module>  File "/tmp/pants-sandbox-sc9D8O/python/liga/registry/__init__.py", line 15, in <module>    from liga.registry.base import command_from_spec  File "/tmp/pants-sandbox-sc9D8O/python/liga/registry/base.py", line 24, in <module>    from rikai.internal.reflection import find_class  File "/tmp/pants-sandbox-sc9D8O/python/rikai/__init__.py", line 19, in <module>    from .__version__ import versionModuleNotFoundError: No module named 'rikai.__version__'
	at ai.eto.rikai.sql.spark.Python$.execute(Python.scala:58)
	at org.apache.spark.sql.rikai.model.ModelResolver$.resolve(ModelResolver.scala:101)
	at ai.eto.rikai.sql.model.PyImplRegistry.resolve(Registry.scala:81)
	at ai.eto.rikai.sql.model.Registry$.resolve(Registry.scala:214)
	at ai.eto.rikai.sql.spark.execution.CreateModelCommand.run(CreateModelCommand.scala:62)
	at org.apache.spark.s

## Step 5: predict using the registered Rikai model

In [ ]:
from liga.numpy.sql import literal

result = spark.sql(f"""
select
  ML_PREDICT(mlflow_sklearn_m, {literal(X[0])})
"""
)

result.printSchema()
result.toPandas()

In [ ]:
spark.sql(f"""
select  ML_PREDICT(mlflow_sklearn_m, {literal(X[1])})
""").toPandas()